In [107]:
from itertools import zip_longest

import pandas as pd

from collections import Counter

In [108]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [127]:
df = pd.read_csv("data/isaiah_dataset_dec_2024.csv").fillna("")
### Exclude MN occurrences
df = df[df.comments != "min excluded"]

### Exclude reconstructed occurrences
df = df[df.comments != "reconstructed"]
df = df[df.comments != "reconstructed?"]

### Exclude occurrences without a complement
df = df[df.complement != "no complement"]

### Exclude occurrences without a complement
df = df[df.lex != "NPL["]

### Keep only factive and fictive motion
df = df[(df.motion_type == "factive") | (df.motion_type == "fictive")]

### Keep only trajectory and goal
df = df[(df.spatial_arg_type == "goal") | (df.spatial_arg_type == "trajectory")]

df = df.reset_index()

df.head(2)

,index,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,spatial_arg_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse
0,6,212418,BW>[,MT,Isaiah,1,23,JBW>,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLMNJM JTWM L> JCPVW W RJB >LMNH L> JBW> >LJHM,,qal,impf,W RJB >LMNH L> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0.0,prep,prps,anim,det,prsf,simple,factive,goal,>L,,,,,
1,7,1895234,BW>[,1Qisaa,Isaiah,1,23,JBW>,FRJKJ SWRRJM W XBRJ GNBJM KWLM >WHBJ CWXD RWDPJ CLMWNJM J>TWM LW> JCPVW W RJB >LMNH LW> JBW> >LJHM,00000 011111 1 1111 00000 0000 00000 0000 00000 0000000 00000 000 00011 1 111 00000 000 0000 00000,qal,impf,W RJB >LMNH LW> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0.0,prep,prps,anim,det,prsf,simple,factive,goal,>L,,,,,FRJKJ S[WRRJM W XBRJ] GNBJM KWLM >WHBJ CWXD RWDPJ CLMWNJM J>TWM LW> JCP[VW W RJB] >LMNH LW> JBW> >LJHM


In [129]:
df.to_csv("data/motion_and_posture_isaiah_dataset_perez.csv", index=False)

In [110]:
Counter(df[df.lex == "<BR["].preposition_1)

Counter({'B': 12, '': 6, '<L': 2})

In [111]:
Counter(df[df.lex == "<BR["].cmpl_constr)

Counter({'prep': 14, 'vc': 4, 'dir-he': 2})

In [112]:
Counter(df[(df.lex == "<BR[") & (df.scroll == "MT")].preposition_1)

Counter({'B': 5, '': 4, '<L': 1})

In [113]:
Counter(df[(df.lex == "<BR[") & (df.scroll == "MT")].cmpl_constr)

Counter({'prep': 6, 'vc': 3, 'dir-he': 1})

In [114]:
Counter(df.lex)

Counter({'BW>[': 117,
         '<LH[': 45,
         'HLK[': 31,
         'CWB[': 29,
         '<BR[': 20,
         'JRD[': 18,
         'QRB[': 10,
         'JY>[': 7,
         'RWY[': 5,
         'XSH[': 4,
         'NGC[': 4,
         'NWS[': 3,
         'NHR[': 2,
         '<WP[': 2,
         'XLP[': 2,
         'PF<[': 2,
         'NWX[': 2,
         'SBB[': 2,
         '<WZ[': 2,
         'CQQ[': 2,
         'CWR=[': 2,
         'T<H[': 1})

In [115]:
df["preposition_1"] = df.preposition_1.str.strip()

In [116]:
set(df.preposition_1)

{'', '<D', '<L', '>L', 'B', 'L'}

In [117]:
print(f"Total: {len(df)}")
print(f"MT: {len(df[df.scroll == 'MT'])}, percentage: {len(df[df.scroll == 'MT'])/ len(df) * 100}")
print(f"1QIsaa: {len(df[df.scroll == '1Qisaa'])}, percentage: {len(df[df.scroll == '1Qisaa'])/ len(df) * 100}")
print(f"1Q8: {len(df[df.scroll == '1Q8'])}, percentage: {len(df[df.scroll == '1Q8'])/ len(df) * 100}")

Total: 312
MT: 144, percentage: 46.15384615384615
1QIsaa: 142, percentage: 45.51282051282051
1Q8: 26, percentage: 8.333333333333332


In [118]:
print(f"""in MT the complements are:\n 
{len(df[(df.scroll == "MT") & (df.cmpl_constr == "prep")])} prepositions,\n
{len(df[(df.scroll == "MT") & (df.cmpl_constr == "vc")])} verbal complements,\n
{len(df[(df.scroll == "MT") & (df.cmpl_constr == "dir-he")])} directive -he,\n
{len(df[(df.scroll == "MT") & (df.cmpl_constr == "prep + dir-he")])} prepositions plus directive -he,\n""")

in MT the complements are:
 
112 prepositions,

26 verbal complements,

6 directive -he,

0 prepositions plus directive -he,



In [119]:
print(f"""in 1QIsaa the complements are:\n 
{len(df[(df.scroll == "1Qisaa") & (df.cmpl_constr == "prep")])} prepositions,\n
{len(df[(df.scroll == "1Qisaa") & (df.cmpl_constr == "vc")])} verbal complements,\n
{len(df[(df.scroll == "1Qisaa") & (df.cmpl_constr == "dir-he")])} directive -he,\n
{len(df[(df.scroll == "1Qisaa") & (df.cmpl_constr == "prep + dir-he")])} prepositions plus directive -he,\n""")

in 1QIsaa the complements are:
 
114 prepositions,

22 verbal complements,

5 directive -he,

1 prepositions plus directive -he,



In [120]:
print(f"""in 1Q8 the complements are:\n 
{len(df[(df.scroll == "1Q8") & (df.cmpl_constr == "prep")])} prepositions,\n
{len(df[(df.scroll == "1Q8") & (df.cmpl_constr == "vc")])} verbal complements,\n
{len(df[(df.scroll == "1Q8") & (df.cmpl_constr == "dir-he")])} directive -he,\n
{len(df[(df.scroll == "1Q8") & (df.cmpl_constr == "prep + dir-he")])} prepositions plus directive -he,\n""")

in 1Q8 the complements are:
 
20 prepositions,

4 verbal complements,

2 directive -he,

0 prepositions plus directive -he,



In [121]:
#df[(df.chapter == 20) & (df.verse_num == 6)]
#df[(df.chapter == 23) & (df.verse_num == 10)]
#df[(df.chapter == 2) & (df.verse_num == 3)]
# + 10:29 ad 60:7

In [122]:
# COMPLEMENT: OCCURRENCE COUNT

cmpl_dic = {}

for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

stats = {}
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, cmpl_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)
            
stats

{('1Qisaa', 'MT'): (135, 3, 138),
 ('1Q8', 'MT'): (26, 0, 26),
 ('1Q8', '1Qisaa'): (25, 1, 26)}

In [123]:
#df[(df.chapter == 52) & (df.verse_num == 11)]

In [124]:
# COMPLEMENT: OCCURRENCE COUNT AND PERCENTAGES, TABLE

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": len(df[df.scroll == scrolls[0]]),
        "total_occ_scroll2": len(df[df.scroll == scrolls[1]]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,1Qisaa,MT,142,144,138,135,3,97.826087
1,1Q8,MT,26,144,26,26,0,100.000000
2,1Q8,1Qisaa,26,142,26,25,1,96.153846


In [74]:
# COMPLEMENT: DISAGREEMENT CASES
cmpl_dic = {}

# Constructing the dictionary from the dataframe
for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

results = []

# Analyzing and comparing complements
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': cmpl_info[0],
                        'book': cmpl_info[1],
                        'chapter': cmpl_info[2],
                        'verse_num': cmpl_info[3],
                        'complement_1': c1,
                        'complement_2': c2
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results
#stats

,scroll,other_scroll,lex,book,chapter,verse_num,complement_1,complement_2
0,1Qisaa,MT,BW>[,Isaiah,10,28,prep + dir-he,prep
1,1Qisaa,MT,<BR[,Isaiah,10,29,prep,vc
2,1Qisaa,MT,<LH[,Isaiah,60,7,prep,vc
3,1Q8,1Qisaa,<LH[,Isaiah,60,7,vc,prep


In [105]:
df[(df.chapter == 10) & (df.verse_num == 29)]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,spatial_arg_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse
188,216019,<BR[,MT,Isaiah,10,29,<BRW,<BRW M<BRH GB< MLWN LNW XRDH H RMH GB<T C>WL NSH,,qal,perf,<BRW M<BRH,,M<BRH,M<BRH/,(in) a pass,0.0,vc,topo,inanim,und,subs,simple,factive,trajectory,,,,,,
190,1898959,<BR[,1Qisaa,Isaiah,10,29,<BR,<BR B M<BRH GB< MLWN LNW XRDH H RMH GB<T C>WL NSH,000 0 00000 000 0000 000 0000 0 000 0000 0000 000,qal,perf,<BR B M<BRH,,B M<BRH,B M<BRH/,in a pass,0.0,prep,topo,inanim,und,subs,simple,factive,trajectory,B,,,,,<BR B M<BRH GB< MLWN LNW XRDH H RMH GB<T C>WL NSH


In [76]:
# PREPOSITIONS: COUNT

prep_dic = {}

for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for i in range(1, 5):
        prep = row[f"preposition_{i}"].strip()
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(preps, other_preps):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, prep_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)

stats

{('1Qisaa', 'MT'): (101, 7, 108),
 ('1Q8', 'MT'): (20, 0, 20),
 ('1Q8', '1Qisaa'): (20, 0, 20)}

In [81]:
# PREPOSITIONS. DISAGREEMENT CASES BETWEEN THE THREE MANUSCRIPTS
prep_dic = {}

# Constructing the dictionary from the dataframe where 'cmpl_constr' is 'prep'
for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for j in range(1, 5):
        prep = row[f"preposition_{j}"].strip()
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

results = []

# Analyzing and comparing prepositions
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            
            for c1, c2 in zip(preps, other_preps):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': prep_info[0],
                        'book': prep_info[1],
                        'chapter': prep_info[2],
                        'verse_num': prep_info[3],
                        'preposition_scroll_1': ", ".join(c1),
                        'preposition_scroll_2': ", ".join(c2),
                        
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results

,scroll,other_scroll,lex,book,chapter,verse_num,preposition_scroll_1,preposition_scroll_2
0,1Qisaa,MT,<LH[,Isaiah,2,3,>L,">L, >L"
1,1Qisaa,MT,BW>[,Isaiah,14,2,">L, >L",>L
2,1Qisaa,MT,BW>[,Isaiah,16,12,<L,>L
3,1Qisaa,MT,HLK[,Isaiah,18,2,L,>L
4,1Qisaa,MT,CWB[,Isaiah,37,7,L,>L
5,1Qisaa,MT,BW>[,Isaiah,59,20,>L,L
6,1Qisaa,MT,BW>[,Isaiah,66,20,>L,<L


In [103]:
df[(df.chapter == 60) & (df.verse_num == 7) & (df.scroll == "1Q8")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,spatial_arg_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse
1168,1925919,<LH[,1Q8,Isaiah,60,7,J<LW,KL Y>N QDR JQBYW LK >JLJ NBJWT JCRTWNK J<LW RYWN MZBXJ W BJT TP>RTJ >P>R,00 000 000 00000 00 0000 00000 0000000 0000 0000 00000 0 000 000000 0000,hifil,impf,J<LW RYWN MZBXJ,,MZBXJ,MZBX/,on my altar,0.0,vc,obj,inanim,det,subs,complex,factive,goal,,,,,,KL Y>N QDR JQBYW LK >JLJ NBJWT JCRTWNK J<LW RYWN MZBXJ W BJT TP>RTJ >P>R


In [87]:
# PREPOSITIONS: MANUSCRIPT AGREEMENT

def total_occ(scroll):
    scroll_preps = prep_dic[scroll]
    total = 0
    for preps in scroll_preps.values():
        total += len(preps)
    return total

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,MT,1Qisaa,112,114,109,109,0,100.000000
1,MT,1Q8,112,20,23,20,3,86.956522
2,1Qisaa,MT,114,112,109,108,1,99.082569
3,1Qisaa,1Q8,114,20,23,20,3,86.956522
4,1Q8,MT,20,112,23,23,0,100.000000
5,1Q8,1Qisaa,20,114,23,23,0,100.000000


In [79]:
# PREPOSITION MISSING

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll == other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            non_missing = 0
            missing = 0
            for c1, c2 in zip_longest(preps, other_preps):
                if c2 is None:
                    missing += 1
                    print(scroll, other_scroll, prep_info, c1, c2)
                else:
                    non_missing += 1
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + non_missing, stat[1] + missing, stat[2] + missing + non_missing)
            
stats

MT 1Q8 ('BW>[', 'Isaiah', 39, 3) ['>L'] None
MT 1Q8 ('BW>[', 'Isaiah', 39, 3) ['>L'] None
MT 1Q8 ('BW>[', 'Isaiah', 47, 9) ['<L'] None
1Qisaa MT ('BW>[', 'Isaiah', 16, 12) ['>L'] None
1Qisaa 1Q8 ('BW>[', 'Isaiah', 39, 3) ['>L'] None
1Qisaa 1Q8 ('BW>[', 'Isaiah', 39, 3) ['>L'] None
1Qisaa 1Q8 ('BW>[', 'Isaiah', 47, 9) ['<L'] None


{('MT', '1Qisaa'): (109, 0, 109),
 ('MT', '1Q8'): (20, 3, 23),
 ('1Qisaa', 'MT'): (108, 1, 109),
 ('1Qisaa', '1Q8'): (20, 3, 23),
 ('1Q8', 'MT'): (23, 0, 23),
 ('1Q8', '1Qisaa'): (23, 0, 23)}

In [80]:
# PREPOSITION MISSING 

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_non_missing": stat[0],
        "total_missing_from_scroll2": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_non_missing,total_missing_from_scroll2,agreement_percentage
0,MT,1Qisaa,112,114,109,109,0,100.000000
1,MT,1Q8,112,20,23,20,3,86.956522
2,1Qisaa,MT,114,112,109,108,1,99.082569
3,1Qisaa,1Q8,114,20,23,20,3,86.956522
4,1Q8,MT,20,112,23,23,0,100.000000
5,1Q8,1Qisaa,20,114,23,23,0,100.000000
